# Reasearch Track II Assignment 2

---

**Author's Name: Omotoye Shamsudeen Adekoya** <br>
**Student ID: 5066348**

---

## NoteBook Description

This Notebook has two functions; it serves as a user interface for the control of non-holonomic robot in a 3D environment and it also show a graphical feedback of the robot state and behaviour. 

## Robot Control

## Feedback Visualization 